## Introduction to Plyto with Keras
#### Python Machine Learning Visualization Toolkit
This notebook will demonstrate how to use our Keras Accuracy Loss function with the PlytoAPI to visualize model accuracy and loss throughout the training process of a machine learning algorithm, as well as a tutorial on how to create your own callback function

<img src='style/icons/machinelearning-blue.svg'> Plyto toolbar item opens the model visualizer for this notebook!

In [ ]:
from plyto import PlytoAPI, KerasAccuracyLossCallback

#### How it works

A callback function that takes a plyto instance as a parameter is passed as a callback to model.fit when using keras for machine learning. A plyto instance requires an Altair spec to define plots. Below is an example of a simple altair spec to create a line graph of samples versus accuracy.

In [ ]:
# an array of altair specs with one plot of samples versus accuracy
spec = [
    {
        # specifies an altair spec
        "$schema": "https://vega.github.io/schema/vega-lite/v2.json",
        "name": "accuracyGraph",
        
        #size of the plot
        "config": {
            "view": {
                "height": 300,
                "width": 300
            }
        },
        
        # name of the dataset as defined in the backend of the callback function
        "data": {
            "name": "dataSet"
        },
        
        # visual encodings of the plot
        "encoding": {
            "x": {
                "field": "samples",
                "type": "quantitative"
            },
            "y": {
                "field": "accuracy",
                "type": "quantitative"
            }
        },
        
        
        "mark": "line"
    }
]

#### Running a model
To demonstrate how plyto works, we will be looking at the MNIST handwritten digit data, which can be loaded from keras.datasets

In [ ]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Convolution2D, MaxPooling2D
from keras.utils import np_utils

# Load pre-shuffled MNIST data into train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()
 
# Preprocess input data
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
 
# Preprocess class labels
Y_train = np_utils.to_categorical(y_train, 10)
Y_test = np_utils.to_categorical(y_test, 10)
 
# Define model architecture
model = Sequential()
 
model.add(Convolution2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(Convolution2D(32, (3, 3), activation='tanh'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
 
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

However you fit your model, simply add the callback as a parameter and open the Plyto model visualizer to see your statistics and plots update.

In [ ]:
# pass the spec into PlytoAPI
plyto_instance = PlytoAPI(spec)

# pass plyto instance to the callback function
callback = KerasAccuracyLossCallback(plyto_instance)

model.fit(X_train, Y_train, batch_size=None, epochs=10, verbose=0, callbacks=[callback])

*Note: if you are to stop and re-run the model, the plyto_instance and callback must be re-initialized. We recommend initializing them in the same cell as the call to model.fit() to ensure this works properly*

#### Writing your own callback function

A callback function for Plyto is a class that takes a keras callback as a parameter. 

Within this custom function, you can define functions to execute at specific points in running the model.

Keras Callbacks
- on_epoch_begin: called at the beginning of every epoch.
- on_epoch_end: called at the end of every epoch.
- on_batch_begin: called at the beginning of every batch.
- on_batch_end: called at the end of every batch.
- on_train_begin: called at the beginning of model training.
- on_train_end: called at the end of model training.

For more information on using callbacks, visit [keras callback documentation](https://keras.io/callbacks/).

For the progress bars in the status bar to work correctly, your callback function must have epochs, sample_amount, total_progress, current_progress, mode, and epoch_number. Further, total_runtime and starttime is required for the panel to display the runtime once the model is complete. Below is a base to work off of, only containing these variables for basic functionality and passing no altair spec for plots.

In [ ]:
from keras.callbacks import Callback
from time import time


class Basic(Callback):

    def __init__(self, plyto_instance):
        self.epochs = None
        self.sample_amount = 0
        self.total_progress = 0
        self.current_progress = 0
        self.mode = 0
        self.total_runtime = 0
        self.starttime = time()
        self.epoch_number = 1
        self.plyto = plyto_instance

    def on_train_begin(self, logs={}):
        """
        Get number of samples/steps per epoch and total number of epochs
        """
        if "samples" in self.params:
            self.sample_amount = self.params["samples"]
        elif "nb_sample" in self.params:
            self.sample_amount = self.params["nb_sample"]
        else:
            self.sample_amount = self.params["steps"]
            self.mode = 1

        self.epochs = self.params["epochs"]
        self.plyto.update_size(self.sample_amount)
        self.plyto.update_total_steps(self.epochs)

    def on_epoch_begin(self, epoch, logs={}):
        """
        Reset the current epoch's progress every new epoch
        """
        self.current_progress = 0

    def on_epoch_end(self, epoch, logs={}):
        self.epoch_number += 1
        self.plyto.update_current_step(self.epoch_number)

    def on_batch_end(self, batch, logs={}):
        """
        Update statistics and datasets
        """
        if self.mode == 0:
            self.current_progress += logs.get("size")
            self.total_progress += logs.get("size")
        else:
            self.current_progress += 1
            self.total_progress += 1
        self.total_runtime = time() - self.starttime
        self.plyto.update_runtime(self.total_runtime)
        self.plyto.update_current_progress(self.current_progress)
        self.plyto.update_total_progress(self.total_progress)
        self.plyto.update_data_set(
            {
                "samples": self.total_progress,
            }
        )
        if self.total_progress % 256 == 0 or self.total_progress == (self.epochs * self.sample_amount):
            self.plyto.send_data()

In [ ]:
basicPlatoInstance = PlytoAPI([])
basicCallback = Basic(basicPlatoInstance)
model.fit(X_train, Y_train, batch_size=None, epochs=2, verbose=0, callbacks=[basicCallback])

Only variables in the data set can be used in the Altair plotting spec and will be displayed in the panel. The basic example only includes the variale "samples", as seen when updating the Plyto data set with

```python
self.plyto.update_data_set({
    "samples": self.total_progress
})
```